In [ ]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [ ]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [ ]:
customer = "ESN"
source = "googleAdwords"
url = return_api_url()
url = "https://api.innkeepr.ai/api"
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [ ]:
sources = call_api_with_accountId(
    f"{url}/sources/query",
    account_id,
    {"name":source},
    logging
)
print(sources)
if len(sources) != 1:
    raise Exception(f"more or less than one source was found: {sources}")
source_id = sources[0]["id"]
print(f"source_id = {source_id}")

In [ ]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"source":source_id},
    logging
)
treatments = pd.json_normalize(treatments)

In [ ]:
treatments.to_csv("treatments.csv")

In [ ]:
treatments["source"].value_counts()

In [ ]:
treatments["properties.type"].value_counts()

In [ ]:
treatments["relates_to.campaign.name"].value_counts()

In [ ]:
if 'relates_to.adset.name' in treatments.columns:
    vc = treatments.groupby(by=["relates_to.campaign.name","relates_to.adset.name","properties.adAccountName"])['id'].value_counts()
elif "relates_to.assetGroup.name" in treatments.columns:
    vc = treatments.groupby(by=["relates_to.campaign.name","relates_to.assetGroup.name","properties.adAccountName"])['id'].value_counts()
else:
    vc = treatments.groupby(by=["relates_to.campaign.name","properties.adAccountName"])['id'].value_counts()
vc = vc.reset_index()
vc

In [ ]:
vc_filtered = treatments[
        # treatments["relates_to.campaign.name"].str.contains("🟠 DE // TOFU // ABO Sales // Creative Testing // ALL Formats // 06.08.2025", case=False).fillna(False) |
        # treatments["relates_to.campaign.name"].str.contains("🟠 DE // TOFU // ABO Sales // Inkeepr Creative Testing // ALL Formats // 15.10.2025", case=False).fillna(False) |
        # treatments["relates_to.campaign.name"].str.contains("🟠🟢 DEAT // TOFU // ABO Sales // Creative Testing // ALL Formats // 21.10.2025", case=False).fillna(False) 
        #treatments["relates_to.campaign.name"].str.contains("🟠🟢 DEAT // TOFU // ABO Sales // Innkeepr Testing // Iced Coffee // Statics // 01.12.2025", case=False).fillna(False)
        treatments["relates_to.campaign.name"].str.contains("VMS", case=False).fillna(False)
        #treatments["relates_to.campaign.name"].str.contains("bottom5p_css", case=False).fillna(False)
    ]
vc_filtered.drop_duplicates(subset=["relates_to.campaign.name"])

In [ ]:
vc_filtered["relates_to.campaign.name"].drop_duplicates().sort_values()

In [ ]:
treatments_filtered=treatments[treatments["id"].isin(vc_filtered["id"].unique().tolist())]
treatments_filtered#["properties.status"].value_counts()

In [ ]:
treatments_list = vc_filtered["id"].unique().tolist()
len(treatments_list), treatments_list


In [ ]:
len(treatments["id"].unique().tolist())

In [ ]:
treatments["created"].min(), treatments["created"].max()

In [ ]:
treatments["id"].unique().tolist()

In [ ]:
externalIds = treatments_filtered["relates_to.assetGroup.id"].dropna().unique().tolist() #4 ['6599333130', '6627040705', '6624995271', '6624995298']
externalIds = treatments_filtered["externalId"].dropna().unique().tolist() #4 ['6599333130', '6627040705', '6624995271', '6624995298']
campaign_name = treatments_filtered["relates_to.campaign.name"].dropna().unique().tolist()
print(campaign_name)
print(len(externalIds), externalIds)
test = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"relates_to.campaign.name":campaign_name},#externalId
    logging
)
test

# Treatments via List

In [ ]:
treatment_file = pd.read_csv("/Users/karolinegriesbach/Downloads/SEA Suchkampagnen DACH.csv",sep=";")
treatment_file

In [ ]:
#treatment_file = treatment_file[treatment_file["Kampagnenname"].str.contains("BRAND", case=False)]
treatment_file

In [ ]:
treatments["relates_to.campaign.name"]

In [ ]:
used_treatments = pd.DataFrame()
for campaign_name in treatment_file["Kampagnenname"].unique().tolist(): #["BRAND"]: #
    temp = treatments[treatments["relates_to.campaign.name"].str.contains(campaign_name, case=False, regex=False)]
    if len(temp) == 0:
        print(f"No treatment found for campaign_name = {campaign_name}")
        continue
    used_treatments = pd.concat([used_treatments, temp])
    print(f"Added treatments for campaign_name = {campaign_name}, used_treatments length = {len(used_treatments)}")


In [ ]:
used_treatments[["id","relates_to.campaign.name"]].to_csv(f"/Users/karolinegriesbach/Downloads/used_treatments_brand_sea_{customer}.csv", index=False)

In [ ]:
temp = treatments[(treatments["relates_to.campaign.name"].str.contains("[P]_SEARCH_BRAND_", regex=False))&(treatments["relates_to.campaign.name"].str.contains(":DE"))].drop_duplicates(subset=["relates_to.campaign.name"]) 
temp["relates_to.campaign.name"].to_csv("treatments_search_brand_de.csv", index=False)
temp